In [1]:
import dropbox
import os
import numpy as np
import pandas as pd
from scipy.stats import t
import math
import collections
from PIL import Image
from dropbox import DropboxOAuth2FlowNoRedirect
import matplotlib.pyplot as plt
import cv2
import pypyodbc
import sys
import traceback
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import BatchNormalization, concatenate, Input, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical  #?
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
cnxn = pypyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=028-GUMUS1-SRV;"
                      "Database=KOTON_DB;"
                      "Trusted_Connection=yes;")

In [3]:
access_token='0DQLcsXIojcAAAAAAAgybeB3DOmXKK7bRTLyYwkthbrAiGmpQR4AuGINQkjBXhif'
dbx = dropbox.Dropbox(access_token)

retrieved_image_df = pd.read_excel('C:/Users/Recep/Koton/Data/retrieved_image_list.xlsx', sheet_name='Sheet1')

In [4]:
def data_set(codes, indexes):
    """Takes list of option codes as input and returns related dataset, response set (sales amount)
    and genuine option codes that included in e-commerce table.
    """
    
    if len(codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
    
    string_tar = "SELECT [OptionCode] \
      ,[Groups] \
  FROM [KOTON_DB].[dbo].[Ratio_table_gruouped] Where ("
    
    codes_split = np.array_split(codes, numOfBatches)
    indexes_split = np.array_split(indexes, numOfBatches)
    
    # to prevent empty targets
    gen_codes = []
    gen_indexes = []
    target = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in codes_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + ')'
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)

    for code, index in zip(codes, indexes):
        try:
            target.loc[code]
            gen_codes.append(code)
            gen_indexes.append(index)
        except:
            pass        
        
    
    string_inp = "SELECT * \
  FROM [KOTON_DB].[dbo].[Input_table] \
  WHERE "
    
#     numOfBatches = max(min(len(gen_indexes), numOfBatches), 1)
    numOfBatches = max(min(len(gen_indexes), numOfBatches), 1)
    gen_codes_splited = np.array_split(gen_codes, numOfBatches)
    dataset = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag1 = True
        for i in gen_codes_splited[batch]:
            if flag1:
                _.append("OptionCode='"+i+"'")
                flag1 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query1 = string_inp + ''.join(_)
        dataset.append(pd.read_sql_query(query1, cnxn, index_col='optioncode'))
    
    dataset = pd.concat(dataset)
    
    
    return dataset, target, gen_codes, gen_indexes

In [ ]:
def tarrain(numberSamples, startIndex, batchSize=64, return_data=False):
    errorLog = []
    bn_model = Xception(include_top=False, weights='imagenet', pooling='max')

    first = bn_model.output
    for layer in bn_model.layers:
        layer.trainable = False

    model = Dense(256, activation='relu')(first)
    model = Dropout(0.5)(model)
    model = Dense(10, activation='softmax')(model)

    model_ = Model(inputs=[bn_model.input], outputs=model)

    model_.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    codes = []
    idx = []
    for picture_index in range(startIndex,startIndex + numberSamples):
        code = retrieved_image_df.iloc[picture_index].name
        codes.append(code)
        idx.append(picture_index)
    
    dset, tar, gen_codes, gen_indexes = data_set(codes, idx)
#     dset = dset.apply(LabelEncoder().fit_transform)
    dset = pd.get_dummies(dset)
    tar = pd.get_dummies(tar)
    print(dset.shape)
    print(tar.shape)
       
    if batchSize>dset.shape[0]:
        batchSize = dset.shape[0]
        print('Batch size is automatically adjusted.')
    
    dset, dset_test, tar, tar_test = train_test_split(dset, tar, test_size=0.01, random_state=42)
    gen_codes, gen_codes_test, gen_indexes, gen_indexes_test = train_test_split(gen_codes, gen_indexes, 
                                                                                test_size=0.01, random_state=42)
    print(dset.shape)
    
    numOfBatches, lastBatch = divmod(dset.shape[0], batchSize-1)
    dsets = np.array_split(dset, numOfBatches)
    tars = np.array_split(tar, numOfBatches)
    gen_codes = np.array_split(gen_codes, numOfBatches)
    gen_indexes = np.array_split(gen_indexes, numOfBatches)
    
    valid_pic1 =[]
    for code, picture_index in zip(gen_codes_test, gen_indexes_test):
        im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
        if (math.isnan(float(im_index))==False):
            filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
            path = '/Koton_Image_Files/'+filename
            try:
                md, res = dbx.files_download(path)
                temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                assert temp_pic.shape == (299, 299, 3)
                valid_pic1.append(temp_pic)
            except:
                errorLog.append(path+' Not found')
    valid_pic1 = preprocess_input(np.array(valid_pic1))
#     valid_bottleneck_features = bn_model.predict(valid_pic1)
    
    
    for epoch in range(60):
        tars2 = []
        preds = []
        print('Epoch: '+str(epoch+1)+'/'+str(60))
        for batch in range(numOfBatches):
            pic1 =[]
            print('Batch: '+str(batch+1)+'/'+str(numOfBatches))
            for code, picture_index in zip(gen_codes[batch], gen_indexes[batch]):
                im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
                if (math.isnan(float(im_index))==False):
                    filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
                    path = '/Koton_Image_Files/'+filename
                    try:
                        md, res = dbx.files_download(path)
                        temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                        assert temp_pic.shape == (299, 299, 3)
                        pic1.append(temp_pic)
                    except:
                        errorLog.append(path+' Not found')
                        batch += 1 #ignore defective batch
            pic1 = preprocess_input(np.array(pic1))
            try:
                model_.fit(pic1, tars[batch], validation_data=(valid_pic1, tar_test), epochs=1)
                p = model_.predict(pic1)
                if p.shape == tars[batch].shape:
                    tars2.append(tars[batch])
                    preds.append(p)
                if epoch%3 == 0 and batch==59:
                    acc = categorical_accuracy(np.concatenate(tars2, axis=0 ), np.concatenate(preds, axis=0 ))
                    timestr = time.strftime("%Y%m%d-%H%M%S")
                    model_.save('model'+timestr+'.h5')
                    print("Train accuracy: " + str(acc))
            except:
                exc_type, exc_value, exc_traceback = sys.exc_info()
                lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
                errorLog.append(''.join('!! ' + line for line in lines))
                
    if return_data:
        return model_, errorLog, dset, dset_test, tar, tar_test, gen_codes, gen_codes_test, gen_indexes, gen_indexes_test
    else:
        return model_, errorLog
                    

In [ ]:
model, errorLog, dset, dset_test, tar, tar_test = tarrain(40000, 1724, 64, return_data=True)

(6144, 193)
(6144, 10)
(6082, 193)
Epoch: 1/60
Batch: 1/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 102s 2s/step - loss: 8.7653 - acc: 0.0938 - val_loss: 8.7834 - val_acc: 0.1935
Batch: 2/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 97s 2s/step - loss: 10.8053 - acc: 0.2188 - val_loss: 8.7477 - val_acc: 0.1935
Batch: 3/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 98s 2s/step - loss: 9.1369 - acc: 0.3281 - val_loss: 10.5885 - val_acc: 0.1774
Batch: 4/96
Batch: 5/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 98s 2s/step - loss: 11.6493 - acc: 0.1094 - val_loss: 9.3989 - val_acc: 0.2419
Batch: 6/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 97s 2s/step - loss: 9.8492 - acc: 0.2344 - val_loss: 8.9693 - val_acc: 0.1774
Batch: 7/96


Batch: 57/96
Batch: 58/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 96s 2s/step - loss: 2.2595 - acc: 0.1587 - val_loss: 2.2381 - val_acc: 0.1774
Batch: 59/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 96s 2s/step - loss: 2.2506 - acc: 0.1270 - val_loss: 2.1846 - val_acc: 0.1452
Batch: 60/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 96s 2s/step - loss: 2.0648 - acc: 0.1587 - val_loss: 2.1559 - val_acc: 0.1774
Train accuracy: Tensor("Cast:0", shape=(2924,), dtype=float32)
Batch: 61/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 96s 2s/step - loss: 2.2899 - acc: 0.2698 - val_loss: 2.2313 - val_acc: 0.2258
Batch: 62/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 96s 2s/step - loss: 2.1884 - acc: 0.1905 - val_loss: 2.2183 - val_acc: 0

64/64 [==============================] - 98s 2s/step - loss: 2.1362 - acc: 0.1250 - val_loss: 2.0750 - val_acc: 0.1935
Batch: 11/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 98s 2s/step - loss: 2.1356 - acc: 0.2344 - val_loss: 2.0929 - val_acc: 0.1613
Batch: 12/96
Batch: 13/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 97s 2s/step - loss: 2.2166 - acc: 0.1406 - val_loss: 2.1323 - val_acc: 0.1613
Batch: 14/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 97s 2s/step - loss: 2.1792 - acc: 0.2188 - val_loss: 2.0807 - val_acc: 0.1452
Batch: 15/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 97s 2s/step - loss: 2.3158 - acc: 0.1094 - val_loss: 2.2101 - val_acc: 0.2258
Batch: 16/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 97s 2s/step - lo

63/63 [==============================] - 97s 2s/step - loss: 2.0682 - acc: 0.2698 - val_loss: 2.1709 - val_acc: 0.1935
Batch: 66/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 96s 2s/step - loss: 2.1745 - acc: 0.2063 - val_loss: 2.1165 - val_acc: 0.2258
Batch: 67/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.0632 - acc: 0.2540 - val_loss: 2.0745 - val_acc: 0.2258
Batch: 68/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 99s 2s/step - loss: 2.2816 - acc: 0.1587 - val_loss: 2.1293 - val_acc: 0.1935
Batch: 69/96
Batch: 70/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.1746 - acc: 0.2698 - val_loss: 2.1587 - val_acc: 0.2097
Batch: 71/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - lo

Batch: 21/96
Batch: 22/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 99s 2s/step - loss: 2.2356 - acc: 0.2188 - val_loss: 2.1165 - val_acc: 0.2097
Batch: 23/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 99s 2s/step - loss: 2.2092 - acc: 0.1406 - val_loss: 2.1155 - val_acc: 0.2097
Batch: 24/96
Batch: 25/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 99s 2s/step - loss: 2.1624 - acc: 0.1719 - val_loss: 2.1120 - val_acc: 0.1935
Batch: 26/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 99s 2s/step - loss: 2.1048 - acc: 0.2188 - val_loss: 2.1355 - val_acc: 0.1613
Batch: 27/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 99s 2s/step - loss: 2.1208 - acc: 0.2188 - val_loss: 2.0861 - val_acc: 0.1935
Batch: 28/96
Train on 64 samples, validate o

...

63/63 [==============================] - 97s 2s/step - loss: 2.2433 - acc: 0.1905 - val_loss: 2.0619 - val_acc: 0.2097
Batch: 39/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.1622 - acc: 0.2222 - val_loss: 2.0738 - val_acc: 0.2097
Batch: 40/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.0706 - acc: 0.2222 - val_loss: 2.0606 - val_acc: 0.1613
Batch: 41/96
Batch: 42/96
Batch: 43/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 102s 2s/step - loss: 2.0797 - acc: 0.2222 - val_loss: 2.0700 - val_acc: 0.1774
Batch: 44/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 152s 2s/step - loss: 2.0136 - acc: 0.1270 - val_loss: 2.0602 - val_acc: 0.1935
Batch: 45/96
Batch: 46/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [======================

63/63 [==============================] - 103s 2s/step - loss: 2.0961 - acc: 0.1746 - val_loss: 2.0684 - val_acc: 0.2097
Batch: 91/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.2650 - acc: 0.1746 - val_loss: 2.0980 - val_acc: 0.2742
Batch: 92/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 102s 2s/step - loss: 2.0536 - acc: 0.2222 - val_loss: 2.0689 - val_acc: 0.2581
Batch: 93/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 99s 2s/step - loss: 2.1240 - acc: 0.2063 - val_loss: 2.0478 - val_acc: 0.2258
Batch: 94/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 106s 2s/step - loss: 2.1114 - acc: 0.3333 - val_loss: 2.0601 - val_acc: 0.1613
Batch: 95/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 102s 2s/step - loss: 1.966

63/63 [==============================] - 101s 2s/step - loss: 2.2146 - acc: 0.2698 - val_loss: 2.1720 - val_acc: 0.2258
Batch: 49/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 98s 2s/step - loss: 2.1661 - acc: 0.2540 - val_loss: 2.0598 - val_acc: 0.1774
Batch: 50/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.0500 - acc: 0.1905 - val_loss: 2.0457 - val_acc: 0.2097
Batch: 51/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.1438 - acc: 0.2222 - val_loss: 2.0632 - val_acc: 0.2258
Batch: 52/96
Batch: 53/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 98s 2s/step - loss: 2.1030 - acc: 0.3333 - val_loss: 2.0618 - val_acc: 0.2419
Batch: 54/96
Batch: 55/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 10

Batch: 2/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 97s 2s/step - loss: 2.1088 - acc: 0.2188 - val_loss: 2.0254 - val_acc: 0.2419
Batch: 3/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 98s 2s/step - loss: 2.0388 - acc: 0.2344 - val_loss: 2.0379 - val_acc: 0.2419
Batch: 4/96
Batch: 5/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 98s 2s/step - loss: 2.0355 - acc: 0.2031 - val_loss: 2.0131 - val_acc: 0.2097
Batch: 6/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 98s 2s/step - loss: 2.0174 - acc: 0.2500 - val_loss: 2.0134 - val_acc: 0.2419
Batch: 7/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 98s 2s/step - loss: 2.0873 - acc: 0.2188 - val_loss: 1.9971 - val_acc: 0.2581
Batch: 8/96
Batch: 9/96
Train on 64 samples, validate on 62 sam

63/63 [==============================] - 98s 2s/step - loss: 1.9653 - acc: 0.2540 - val_loss: 2.0408 - val_acc: 0.1935
Batch: 60/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.0025 - acc: 0.2857 - val_loss: 2.0536 - val_acc: 0.1935
Train accuracy: Tensor("Cast_2:0", shape=(2924,), dtype=float32)
Batch: 61/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.0332 - acc: 0.2540 - val_loss: 2.0390 - val_acc: 0.2742
Batch: 62/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.0545 - acc: 0.2698 - val_loss: 2.0380 - val_acc: 0.2097
Batch: 63/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 97s 2s/step - loss: 2.0247 - acc: 0.3016 - val_loss: 2.0612 - val_acc: 0.2097
Batch: 64/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63